In [13]:
# EXP-real-hardware.ipynb

class RealHardware(Experiment):
    pass

    def __init__(self, hamiltonian, backend, experiment_name=""):
        
        super().__init__(hamiltonian, ExperimentType.REAL_HARDWARE, experiment_name)
        self.backend = backend            

    def run(self, runs, mitigate=False):
        
        self.mitigate = mitigate
        if(self.mitigate):
            meas_filter = get_meas_filter(self.backend)
        
        for i in range(runs):

            probabilities_0, probabilities_1 = [], []
            qcs = []

            for T in self.timeline:
                qc = prepare_qc(self.hamiltonian.matrix, T)
                qcs.append(qc)

            job = execute(qcs, self.backend, shots=num_shots_IBMQ)
            print('Job ID', job.job_id())
            
            counts_all = job.result().get_counts()
            
            for i, counts in enumerate(counts_all):
                
                results = job.result()
                counts = results.get_counts(i)
                
                # Results with mitigation
                if(self.mitigate):
                    mitigated_results = meas_filter.apply(results)
                    counts = mitigated_results.get_counts(i)
                
                try:
                    probabilities_0.append(counts['0'])
                    print(".", end="")
                except:
                    # print("An exception occurred & resolved")
                    probabilities_0.append(0)
                    print(":", end="")

                try:
                    probabilities_1.append(counts['1'])
                    print(".", end="")
                except:
                    # print("An exception occurred & resolved")
                    probabilities_1.append(0)
                    print(":", end="")

            print("")

            self.probs[0].append(probabilities_0)
            self.probs[1].append(probabilities_1)
        
        labels = ["run-"+ str(i) for i in range(1,runs+1)]
        
        self.probs = [
            pd.DataFrame(
                list(np.array(self.probs[0]) / num_shots), 
                index=labels, 
            ).round(round_dec),
            pd.DataFrame(
                list(np.array(self.probs[1]) / num_shots), 
                index=labels, 
            ).round(round_dec)
        ]
        
        
        self.probs[0] = self.probs[0].append(pd.Series(self.probs[0].mean(axis=0), name="mean"))
        self.probs[1] = self.probs[1].append(pd.Series(self.probs[1].mean(axis=0), name="mean"))
        
        error_0, error_1 = [], []
        
        for i in range(steps):
            error_0.append(mae(self.probs[0].iloc[0:-1, i], [self.probs[0].loc["mean", i]]*runs))
            error_1.append(mae(self.probs[1].iloc[0:-1, i], [self.probs[1].loc["mean", i]]*runs))
            
        self.probs[0] = self.probs[0].append(pd.Series(error_0, name="error"))
        self.probs[1] = self.probs[1].append(pd.Series(error_1, name="error"))
        self.probs[0].columns =["point-"+ str(i) for i in range(1,steps+1)]
        self.probs[1].columns =["point-"+ str(i) for i in range(1,steps+1)]
        self.data = pd.concat(self.probs, keys=["ZERO", "ONE"]) 
        
        print("---\n> experiment " + self.experiment_name + " successfully finished \U0001F389\n---\n")

    def export_data(self, name):
        self.probs[0].to_csv('../OUTPUT/DATA/IBMQ/' + name + '-0.csv', encoding='utf-8')
        self.probs[1].to_csv('../OUTPUT/DATA/IBMQ/' + name + '-1.csv', encoding='utf-8')
        self.data.to_csv('../OUTPUT/DATA/IBMQ/' + name + '.csv', encoding='utf-8')
    
    def plot(self, theory_data, zero=True, one=True, theme='_classic_test_patch', save=[False, ""]):
        
        H1_theoretical_analyzis = [
            pd.read_csv("../OUTPUT/DATA/THEORY/" + theory_data + "-0.csv", index_col="Unnamed: 0"),
            pd.read_csv("../OUTPUT/DATA/THEORY/" + theory_data + "-1.csv", index_col="Unnamed: 0")
        ]
        
        with plt.style.context(theme):
            plt.xlabel("time [fs]")
            plt.ylabel("probability of measuring states |0⟩ and |1⟩")

            if(zero):
                plt.plot(
                    self.timeplot * 10**15,
                    H1_theoretical_analyzis[0].loc["run-1"],
                    color='gray', linestyle='dashed', linewidth=1,
                    label="computed probability of measuring |0⟩",
                )
                
                plt.errorbar(
                    self.timeplot * 10**15, 
                    self.probs[0].loc["mean"], 
                    self.probs[0].loc["error"], 
                    elinewidth=1, ecolor='red', 
                    lolims=False, capsize=3, capthick=1,
                    color='blue', linestyle='', linewidth=1.5, 
                    marker='+', markersize=3,
                    label="mean probability of measuring |0⟩",
                )
                
            if(one):
                plt.plot(
                    self.timeplot * 10**15,
                    H1_theoretical_analyzis[1].loc["run-1"],
                    color='gray', linestyle='dashed', linewidth=1,
                    label="computed probability of measuring |1⟩",
                )
            
                plt.errorbar(
                    self.timeplot * 10**15, 
                    self.probs[1].loc["mean"], 
                    self.probs[1].loc["error"], 
                    elinewidth=1, ecolor='red', 
                    lolims=False, capsize=3, capthick=1,
                    color='orange', linestyle='', linewidth=1.5, 
                    marker='+', markersize=3,
                    label="mean probability of measuring |1⟩",
                )

        plt.title("Evolution of states |0⟩ and |1⟩ under H")
        plt.legend(loc="best")
        plt.grid()

        if(save[0]):
            plt.savefig('../OUTPUT/PLOTS/IBMQ/' + save[1] + '.png')
            
        plt.show()

---

In [1]:
# print("... running of {Experiment.ipynb} successfully finished \U0001F389")